# Movie creator for ROMS-CROCO parameters
##### Plots figures to create a video with ffmpeg

In [34]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter as gauss
# from scipy.integrate import trapezoid as trapz
from pyspec import spectrum as spec
from matplotlib.colors import LogNorm
import warnings
import scipy.signal as sig
import cmocean as cm

import sys
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle


from dask.diagnostics import ProgressBar


import scipy.integrate as integ
warnings.filterwarnings("ignore")

In [7]:

path='/data/pacific/lrenault/SASI/CROCO/FULL/'
listm=glob(path+'surf*.nc');listm.sort()
# varf=xr.open_dataset(listm[-9:][xmon])
varF=xr.open_mfdataset(listm[1:],data_vars='minimal')

path='/data/pacific/lrenault/SASI/CROCO/SMTH/'
listt=glob(path+'surf*SASI_6h_his_20*_*.nc');listt.sort()
listt.append('/data/pacific/lrenault/SASI/CROCO/SMTH/surf_SASI_6h_his_2012-07.nc')
#vart=xr.open_dataset(listt[xmon])
varT=xr.open_mfdataset(listt[1:],data_vars='minimal')

In [8]:

chunk={'time_counter':20}
path='/data/pacific/lrenault/SASI/FULL/WRF/'
listm=glob(path+'*DAY.nc');listm.sort()
# varf=xr.open_dataset(listm[-9:][xmon])
varf=xr.open_mfdataset(listm[-7:],chunks=chunk)


path='/data/pacific/lrenault/SASI/SMTH/WRF/'
listt=glob(path+'*DAY.nc');listt.sort()
#vart=xr.open_dataset(listt[xmon])
vart=xr.open_mfdataset(listt[2:],chunks=chunk)

In [3]:
plt.rcParams['font.size']=15
plt.rcParams['font.sans-serif']='Arial'
plt.rcParams['font.weight']='bold'

In [4]:
import datetime

# t_ind=pd.DatetimeIndex(varf.time_counter.compute())
t_ind=varf.time_counter.groupby("time_counter.dayofyear").mean()
mon_ind=t_ind.dt.month
mes=np.array([12,1,2,3,4,5,6,7])
mes_str=['Dec/11','Jan/12','Feb/12','Mar/12','Apr/12','May/12','Jun/12','Jul/12']

In [6]:
varf

<xarray.Dataset>
Dimensions:              (y_rho: 1520, x_rho: 1000, y_u: 1520, x_u: 999, y_v: 1519, x_v: 1000, y_w: 1520, x_w: 1000, time_counter: 852, axis_nbounds: 2)
Coordinates:
    nav_lat_rho          (y_rho, x_rho) float32 dask.array<chunksize=(1520, 1000), meta=np.ndarray>
    nav_lon_rho          (y_rho, x_rho) float32 dask.array<chunksize=(1520, 1000), meta=np.ndarray>
    nav_lat_u            (y_u, x_u) float32 dask.array<chunksize=(1520, 999), meta=np.ndarray>
    nav_lon_u            (y_u, x_u) float32 dask.array<chunksize=(1520, 999), meta=np.ndarray>
    nav_lat_v            (y_v, x_v) float32 dask.array<chunksize=(1519, 1000), meta=np.ndarray>
    nav_lon_v            (y_v, x_v) float32 dask.array<chunksize=(1519, 1000), meta=np.ndarray>
    nav_lat_w            (y_w, x_w) float32 dask.array<chunksize=(1520, 1000), meta=np.ndarray>
    nav_lon_w            (y_w, x_w) float32 dask.array<chunksize=(1520, 1000), meta=np.ndarray>
    time_instant         (time_counter) datetime64[ns] dask.array<chunksize=(124,), meta=np.ndarray>
  * time_counter         (time_counter) datetime64[ns] 2012-01-01T06:00:00 .....
Dimensions without coordinates: y_rho, x_rho, y_u, x_u, y_v, x_v, y_w, x_w, axis_nbounds
Data variables: (12/43)
    s_rho                float32 -0.00625
    s_w                  float32 0.0
    time_instant_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(124, 2), meta=np.ndarray>
    time_counter_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(124, 2), meta=np.ndarray>
    hc                   float64 200.0
    theta_s              float64 6.0
    ...                   ...
    v                    (time_counter, y_v, x_v) float32 dask.array<chunksize=(124, 1519, 1000), meta=np.ndarray>
    w                    (time_counter, y_rho, x_rho) float32 dask.array<chunksize=(124, 1520, 1000), meta=np.ndarray>
    omega                (time_counter, y_w, x_w) float32 dask.array<chunksize=(124, 1520, 1000), meta=np.ndarray>
    temp                 (time_counter, y_rho, x_rho) float32 dask.array<chunksize=(124, 1520, 1000), meta=np.ndarray>
    salt                 (time_counter, y_rho, x_rho) float32 dask.array<chunksize=(124, 1520, 1000), meta=np.ndarray>
    rho                  (time_counter, y_rho, x_rho) float32 dask.array<chunksize=(124, 1520, 1000), meta=np.ndarray>
Attributes: (12/49)
    name:                      Out/AVG_Y2012M01/SASI_6h_his
    description:               Created by xios
    Conventions:               CF-1.6
    timeStamp:                 2020-Jul-30 22:12:58 GMT
    uuid:                      1e6e6c86-13c6-4527-ac3f-51b9c243770b
    title:                     SASI
    ...                        ...
    sponge_expl:               Sponge parameters : extent (m) & viscosity (m2...
    SRCS:                      main.F step.F read_inp.F timers_roms.F init_sc...
    CPP-options:               REGIONAL SASI DIAGNOSTICS_EDDY DIAGNOSTICS_TS ...
    history:                   Sun Apr 17 12:57:27 2022: ncwa -a s_rho tmp2.n...
    NCO:                       "4.6.0"
    nco_openmp_thread_number:  1

In [56]:
'WRF'
chunks_new={'time_counter':40}


wf=varf.WSPD10
wf=wf.chunk(chunks=chunks_new).compute()
wt=vart.WSPD10.compute()
wt=wt.chunk(chunks=chunks_new).compute()


t2f=varf.T2-273.5
t2f=t2f.chunk(chunks=chunks_new).compute()
t2t=vart.T2-273.5
t2t=t2t.chunk(chunks=chunks_new).compute()


uf=varf.UOCE
uf=uf.chunk(chunks=chunks_new).compute()
vf=varf.VOCE
vf=vf.chunk(chunks=chunks_new).compute()

ut=vart.UOCE
ut=ut.chunk(chunks=chunks_new).compute()
vt=vart.VOCE
vtf=vt.chunk(chunks=chunks_new).compute()



In [28]:
speedf=np.sqrt(uf**2 + vf**2)
speedt=np.sqrt(ut**2 + vt**2)

In [22]:
%%time
'CROCO'
chunks_new={'dayofyear':40}

tf=varF.temp.groupby("time_counter.dayofyear").mean()
tf=tf.chunk(chunks=chunks_new).compute();

tt=varT.temp.groupby("time_counter.dayofyear").mean()
tt=tt.chunk(chunks=chunks_new).compute();

# uf=varF.u.groupby("time_counter.dayofyear").mean()
# uf=uf.chunk(chunks=chunks_new).compute();

# ut=varT.u.groupby("time_counter.dayofyear").mean()
# ut=ut.chunk(chunks=chunks_new).compute();

# vf=varF.v.groupby("time_counter.dayofyear").mean()
# vf=vf.chunk(chunks=chunks_new).compute();

# vt=varT.v.groupby("time_counter.dayofyear").mean()
# vt=vt.chunk(chunks=chunks_new).compute();



CPU times: user 30.7 s, sys: 7.77 s, total: 38.4 s
Wall time: 4.25 s


In [93]:
mask

array([[286.19046, 286.1899 , 286.1888 , ...,   0.     ,   0.     ,
          0.     ],
       [286.19046, 286.19046, 286.1899 , ...,   0.     ,   0.     ,
          0.     ],
       [286.19046, 286.19046, 286.19046, ...,   0.     ,   0.     ,
          0.     ],
       ...,
       [285.70343, 285.70224, 285.6998 , ...,   0.     ,   0.     ,
          0.     ],
       [285.70224, 285.69977, 285.69977, ...,   0.     ,   0.     ,
          0.     ],
       [285.69977, 285.69977, 285.69974, ...,   0.     ,   0.     ,
          0.     ]], dtype=float32)

In [100]:
mask=varf.SST[0].to_numpy().copy()
mask[varf.SST[0]!=0]=np.nan;
# plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)


In [127]:
wf.mean()

<xarray.DataArray 'WSPD10' ()>
array(6.9327574, dtype=float32)

In [169]:
# id=0
for id in range(varf.SST.shape[0]):
    plt.figure(figsize=(16,6))
    plt.subplot(2,4,1)

    speedf[id].plot(x='nav_lon',y='nav_lat',
                                 cmap=cm.cm.speed,levels=np.linspace(0,0.8,55),
              cbar_kwargs={'ticks':np.arange(0,1,0.2),'label':r'$m \,s^{-1} $'},extend='max')
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)
    plt.title(r'{0} $Speed$ (FULL)'.format(pd.DatetimeIndex(varf.time_counter)[id]),fontsize=13)


    plt.xlabel('')
    plt.ylabel('')

    plt.subplot(2,4,2)

    speedt[id].plot(x='nav_lon',y='nav_lat',
                                 cmap=cm.cm.speed,levels=np.linspace(0,0.8,55),
              cbar_kwargs={'ticks':np.arange(0,1,0.2),'label':r'$m \,s^{-1} $'},extend='max')
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)
    plt.title(r' $Speed$ (SMTH)',fontsize=13)


    plt.xlabel('')
    plt.ylabel('')

    plt.subplot(2,4,3)

    wf[id].plot(x='nav_lon',y='nav_lat',
                                 cmap='spring_r',levels=np.linspace(2,15,55),
              cbar_kwargs={'ticks':np.arange(0,15,2),'label':r'$m \,s^{-1} $'},extend='both')
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)
    plt.title(r'$Wind$ (FULL)',fontsize=13)

    plt.xlabel('')
    plt.ylabel('')

    plt.subplot(2,4,4)

    wt[id].plot(x='nav_lon',y='nav_lat',
                                 cmap='spring_r',levels=np.linspace(2,15,55),
              cbar_kwargs={'ticks':np.arange(0,15,2),'label':r'$m \,s^{-1} $'},extend='both')
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)
    plt.title(r'$Wind$ (SMTH)',fontsize=13)

    plt.xlabel('')
    plt.ylabel('')

    plt.subplot(2,4,5)

    tf[id].plot(x='nav_lon_rho',y='nav_lat_rho',
                                 cmap='rainbow',levels=np.linspace(9,15,55),
              cbar_kwargs={'ticks':np.arange(5,20,2),'label':r'$^{\circ}C $'},extend='both')
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)
    plt.title(r'$SST$ (FULL)',fontsize=13)

    plt.xlabel('')
    plt.ylabel('')

    plt.subplot(2,4,6)

    tt[id].plot(x='nav_lon_rho',y='nav_lat_rho',
                                 cmap='rainbow',levels=np.linspace(9,15,55),
              cbar_kwargs={'ticks':np.arange(5,20,2),'label':r'$^{\circ}C $'},extend='both')
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)
    plt.title(r'$SST$ (SMTH)',fontsize=13)

    plt.xlabel('')
    plt.ylabel('')


    plt.subplot(2,4,7)

    t2f[id].plot(x='nav_lon',y='nav_lat',
                                 cmap='magma',levels=np.linspace(5,15,55),
              cbar_kwargs={'ticks':np.arange(5,20,4),'label':r'$^{\circ}C $'})
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)
    plt.title(r'$T_{air}$ (FULL)',fontsize=13)

    plt.xlabel('')
    plt.ylabel('')

    plt.subplot(2,4,8)

    t2t[id].plot(x='nav_lon',y='nav_lat',
                                 cmap='magma',levels=np.linspace(5,15,55),
              cbar_kwargs={'ticks':np.arange(5,20,4),'label':r'$^{\circ}C $'})
    plt.contourf(varf.SST[0].nav_lon,varf.SST[0].nav_lat,mask)


    plt.title(r'$T_{air}$ (SMTH)',fontsize=13)

    plt.xlabel('')
    plt.ylabel('')
    plt.tight_layout()
    plt.savefig('/homes/metogra/iufarias/Documents/figures/movies/ocean+atm_T+vel_movies_{0}.png'.format(pd.DatetimeIndex(varf.time_counter)[id]))
    plt.close()